In [1]:
import os
os.environ["KERAS_BACKEND"] = "theano"

from __future__ import print_function
import numpy as np

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import PIL
from PIL import Image, ImageFilter
import requests
from io import BytesIO

import imp

Using Theano backend.


In [2]:
#persona definition
persona_definition_path = os.path.abspath(os.path.join('..', '..', 'Personas', 'personaDefinition_pb2.py'))
print (persona_definition_path)
#import persona proto modules
persona = imp.load_source('Persona', persona_definition_path).Persona()

#load persona
personaName = "Khandhasamy"
personaCategory = ['Artist', 'Portraits', 'sketchToGreyImage'] 
personaPath = os.path.join('..', '..', 'Personas')
for category in personaCategory:
    personaPath = os.path.join(personaPath, category)

#persona file
persona_abs_path = os.path.abspath(os.path.join(personaPath, personaName , personaName + '.bin'))
f = open(persona_abs_path, "rb")
persona.ParseFromString(f.read())

def getInformationSource(persona, layer):
    for environment in persona.age.environments:
        for information in environment.informations:
            for informationConnectedLayerName in information.connectedLayerName:
                if (layer.layerName == informationConnectedLayerName):
                    #print (information.informationSource)
                    return getExtractor(information)

def getExtractor(information):
    #information definition
    information_definition_path = os.path.abspath(os.path.join('..', '..', 'Environment', information.informationSource + "_pb2.py"))
    informationModule = imp.load_source('Information', information_definition_path).Information()  
    information_file_path = os.path.abspath(os.path.join('..', '..', 'Environment', information.informationSource + ".bin"))
    informationFile = open(information_file_path, "rb")
    informationModule.ParseFromString(informationFile.read())
    #load extractor
    information_extractor_path = os.path.abspath(os.path.join('..', '..', 'Environment/Informations/Process/Extract/' + informationModule.extractor + ".py"))
    extractorModule = imp.load_source(informationModule.extractor, information_extractor_path).ImageURLExtractor(informationModule)
    #print (informationModule.extractor)
    return extractorModule

LAYER_CONVOLUTION = "layerConvolution"
LAYER_ACTIVATION = "layerActivation"
LAYER_DROPOUT = "layerDropout"

model = Sequential()

def runPersona(persona):
    x_train_data = None
    y_train_data = None
    x_action_data = None
    input_shape = None
    batch_size = None

    for dna in persona.DNAs:
        for inputLayer in dna.inputs:
            print (inputLayer.layerName)
            inputTransform = inputLayer.inputTransform
            #get information source
            information = getInformationSource(persona, inputLayer)

            x_train_data = information.getData(inputTransform)
            #x_action_data = information.get_data(inputTransform)
            print ("input")
            
        for layer in dna.layers:
            if (LAYER_CONVOLUTION == layer.WhichOneof("SubLayer")):
                nb_filters = layer.layerConvolution.filters
                convDimension = layer.layerConvolution.convolutionDimension
                borderMode = layer.layerConvolution.borderMode
                kernelSize = layer.layerConvolution.kernelSize
                inputShape = layer.layerConvolution.inputShape
                if K.image_dim_ordering() == 'th':
                    conv_input_shape = (inputShape[0], inputShape[1], inputShape[2])
                else:
                    conv_input_shape = (inputShape[1], inputShape[2], inputShape[0])
                if convDimension == 2:
                    model.add(Convolution2D(nb_filters, kernelSize[0], kernelSize[1],
                        border_mode=borderMode,
                        input_shape=conv_input_shape))
                    print (conv_input_shape)
                    print ("conv2D")
            if (LAYER_ACTIVATION == layer.WhichOneof("SubLayer")):  
                activationType = layer.layerActivation.activationType
                model.add(Activation(activationType))
                print ("activation")
            if (LAYER_DROPOUT == layer.WhichOneof("SubLayer")):
                dropoutPercentage = layer.layerDropout.dropPercentage
                model.add(Dropout(dropoutPercentage))
                print ("dropout")
                
        for outputLayer in dna.outputs:
            #print (outputLayer.layerName)
            inputTransform = outputLayer.inputTransform
            #get information source
            information = getInformationSource(persona, outputLayer)
            #train data
            y_train_data = information.getData(inputTransform)
            print ("output")
    
    #compile model
    model.compile(loss='binary_crossentropy', optimizer='adadelta')
    
    print (x_train_data.shape)
    print (y_train_data.shape)

    model.summary()
    
    #learning
    batch_size = persona.age.learningBatchSize
    print (batch_size)
    nb_epoch = 50
    model.fit(x_train_data, y_train_data, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(x_train_data, y_train_data))
    
    #output for given input
    #output_data = model.predict(x_action_data)
    
runPersona(persona)



C:\Users\rames\Documents\GitHub\ai-personas\Personas\personaDefinition_pb2.py
imageInput
(50L, 50L)
http://www.loc.gov/rr/scitech/subjectguides/images/tesla-new.jpg
(50L, 50L)
https://static1.squarespace.com/static/51cdd10de4b08819bd7bc9b4/525d89c2e4b0f8245cabfc96/53cfd660e4b089801fe7e3d9/1458152617516/SciSource_BK0929.jpg?format=750w
(50L, 50L)
https://static1.squarespace.com/static/51cdd10de4b08819bd7bc9b4/525d89c2e4b0f8245cabfc96/53cfd28be4b0c47638e2f540/1458152692173/SciSource_BN4337.jpg?format=750w
input
(1L, 50L, 50L)
conv2D
activation


WARNING (theano.gof.compilelock): Overriding existing lock by dead process '7044' (I am process '644')


dropout
(1L, 50L, 50L)
conv2D
activation
dropout
(1L, 50L, 50L)
conv2D
activation
(50L, 50L)
http://www.loc.gov/rr/scitech/subjectguides/images/tesla-new.jpg
(50L, 50L)
https://static1.squarespace.com/static/51cdd10de4b08819bd7bc9b4/525d89c2e4b0f8245cabfc96/53cfd660e4b089801fe7e3d9/1458152617516/SciSource_BK0929.jpg?format=750w
(50L, 50L)
https://static1.squarespace.com/static/51cdd10de4b08819bd7bc9b4/525d89c2e4b0f8245cabfc96/53cfd28be4b0c47638e2f540/1458152692173/SciSource_BN4337.jpg?format=750w
output
(3L, 1L, 50L, 50L)
(3L, 1L, 50L, 50L)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 100L, 50L, 50L 1000        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 100L, 50L